In [3]:
import sys
import os
import subprocess
from math import *
import numpy as np
from datetime import *
from pylab import *
import matplotlib as mpl
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.pyplot as plt
from astropy.table import Table, Column 
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as patches
import scipy.misc as scimisc
import random
from datetime import datetime
import json

# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# !pip install imgaug

import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib


In [5]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa


ia.seed(100)

seq = iaa.Sequential([

    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.1))),
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 3), per_channel=0.5),
    iaa.Add((-5, 5), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    iaa.Invert(0.05, per_channel=True),
    iaa.Sometimes(0.1, iaa.Grayscale(alpha=(0.0, 1.0))),
    iaa.Sometimes(0.50, iaa.Grayscale(alpha=1.)),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})
    ], random_order=True)


seqGray = iaa.Sequential([

    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.1))),
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 3), per_channel=0.5),
    iaa.Add((-5, 5), per_channel=0.5),
    iaa.Invert(0.5, per_channel=False),
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})
    ], random_order=True)

In [ ]:
data = np.load('./data_128x128_RGB.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
data.close() 
del data

In [5]:
from sklearn.model_selection import train_test_split

ix, = np.where(labels_[1]>45)

images_c = images_[ix]
labels_c = labels_[1][ix]
pgcIDs_c = labels_[0][ix]

images_train, images_test, labels_train, labels_test, pgcIDs_train,  pgcIDs_test = train_test_split(images_c, labels_c, pgcIDs_c, test_size=0.40)

notUnique = np.isin(pgcIDs_test, pgcIDs_train)
Unique = np.logical_not(notUnique)

images_train_RGB = np.concatenate((images_train, images_test[notUnique]))
labels_train_RGB  = np.concatenate((labels_train, labels_test[notUnique]))
pgcIDs_train_RGB = np.concatenate((pgcIDs_train, pgcIDs_test[notUnique]))

images_test_RGB = images_test[Unique]
labels_test_RGB = labels_test[Unique]
pgcIDs_test_RGB = pgcIDs_test[Unique]

del images_
del labels_

del images_c
del labels_c
del pgcIDs_c

In [6]:
data = np.load('./data_128x128_g.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
data.close() 
del data

pgcIDs_ = labels_[0]
labels_ = labels_[1]

inTrain = np.isin(pgcIDs_, pgcIDs_train_RGB)
inTest = np.isin(pgcIDs_, pgcIDs_test_RGB)

images_train_gray = images_[inTrain]
labels_train_gray = labels_[inTrain]
pgcIDs_train_gray = pgcIDs_[inTrain]

images_test_gray = images_[inTest]
labels_test_gray = labels_[inTest]
pgcIDs_test_gray = pgcIDs_[inTest]


del pgcIDs_
del labels_
del images_

In [7]:
data = np.load('./data_128x128_r.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
data.close() 
del data

pgcIDs_ = labels_[0]
labels_ = labels_[1]

inTrain = np.isin(pgcIDs_, pgcIDs_train_RGB)
inTest = np.isin(pgcIDs_, pgcIDs_test_RGB)

images_train_gray = np.concatenate((images_train_gray, images_[inTrain]))
labels_train_gray = np.concatenate((labels_train_gray, labels_[inTrain]))
pgcIDs_train_gray = np.concatenate((pgcIDs_train_gray, pgcIDs_[inTrain]))

images_test_gray = np.concatenate((images_test_gray, images_[inTest]))
labels_test_gray = np.concatenate((labels_test_gray, labels_[inTest]))
pgcIDs_test_gray = np.concatenate((pgcIDs_test_gray, pgcIDs_[inTest]))


del pgcIDs_
del labels_
del images_

In [9]:
data = np.load('./data_128x128_i.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
data.close() 
del data

pgcIDs_ = labels_[0]
labels_ = labels_[1]

inTrain = np.isin(pgcIDs_, pgcIDs_train_RGB)
inTest = np.isin(pgcIDs_, pgcIDs_test_RGB)

images_train_gray = np.concatenate((images_train_gray, images_[inTrain]))
labels_train_gray = np.concatenate((labels_train_gray, labels_[inTrain]))
pgcIDs_train_gray = np.concatenate((pgcIDs_train_gray, pgcIDs_[inTrain]))

images_test_gray = np.concatenate((images_test_gray, images_[inTest]))
labels_test_gray = np.concatenate((labels_test_gray, labels_[inTest]))
pgcIDs_test_gray = np.concatenate((pgcIDs_test_gray, pgcIDs_[inTest]))

del pgcIDs_
del labels_
del images_

In [11]:
N_testGray = len(pgcIDs_test_gray)
N_trainGray = len(pgcIDs_train_gray)

N_testGray, N_trainGray

(3384, 129060)

In [13]:
indx = np.arange(N_testGray)
indx = esn_shuffle(indx, seed=0)

images_test_gray = images_test_gray[indx]
labels_test_gray = labels_test_gray[indx]
pgcIDs_test_gray = pgcIDs_test_gray[indx]

In [14]:
indx = np.arange(N_trainGray)
indx = esn_shuffle(indx, seed=0)

images_train_gray = images_train_gray[indx]
labels_train_gray = images_train_gray[indx]
pgcIDs_train_gray = images_train_gray[indx]

In [15]:
np.savez_compressed('./Gray_128x128_train_000.npz', 
                    images=images_train_gray, 
                    labels=labels_train_gray, 
                    pgcIDs=pgcIDs_train_gray)
np.savez_compressed('./Gray_128x128_test_000.npz', 
                    images=images_test_gray, 
                    labels=labels_test_gray, 
                    pgcIDs=pgcIDs_test_gray)

In [16]:
np.savez_compressed('./RGB_128x128_train_000.npz', 
                    images=images_train_RGB, 
                    labels=labels_train_RGB, 
                    pgcIDs=pgcIDs_train_RGB)
np.savez_compressed('./RGB_128x128_test_000.npz', 
                    images=images_test_RGB, 
                    labels=labels_test_RGB, 
                    pgcIDs=pgcIDs_test_RGB)

### Run this part separately .....

In [19]:
def esn_shuffle(array, seed=0):
        np.random.seed(seed)
        np.random.shuffle(array)
        return array

In [ ]:
# data = np.load('./Gray_128x128_train_000.npz')
# images_ = data['images'].astype(np.uint8)
# labels_ = data['labels'] 
# # data.close() 
# del data

In [16]:
data = np.load('./Gray_128x128_train_000.npz')
images_ = data['images'].astype(np.uint8)
labels_ = data['labels'] 
pgcIDs_ = labels_[0]
labels_ = labels_[1]
data.close() 
del data

In [17]:
N_trainRGB = 42030
N_testRGB = 1128
N_trainGray = 129060
N_testGray = 3384

In [ ]:
for i in range(1):
    
    # RGB
#     images_train_aug = seq(images=images_) 
#     labels_train_aug = labels_
#     pgcIDs_train_aug = pgcIDs_

    # Gray
    indx = np.arange(N_trainGray)
    indx = esn_shuffle(indx, seed=i+10)
    images_train_aug = seqGray(images=images_[indx][:N_trainRGB,:,:,:])
    labels_train_aug = labels_[indx][:N_trainRGB]
    pgcIDs_train_aug = pgcIDs_[indx][:N_trainRGB]

    t2 =  datetime.now()
    print("Execution time: ", i, t2-t1)

    np.savez_compressed('./Gray_128x128_train_aug_'+str(i)+'.npz', images=images_train_aug, labels=labels_train_aug, pgcIDs=pgcIDs_train_aug)
    print('save #', j)

In [ ]:
n = 10
m = 10

fig, axes = plt.subplots(nrows=n, ncols=m, figsize=(15,15)) 

t = 0 
for i in range(n):
    for j in range(m):

        ax = axes[i][j]
        ax.imshow(images_train_aug[t,:,:])

        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)

        t+=1
  
plt.subplots_adjust(hspace=0.03, wspace=0.03)